<a href="https://colab.research.google.com/github/atharv946/Chromatin-Accessibility/blob/main/Step9a_Transformer_based_embeding_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/joined_train_random_1k.csv')

df_val = pd.read_csv('/content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/joined_valid_random_1k.csv')

df_test = pd.read_csv('/content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/joined_test_random_1k.csv')

In [ ]:
# Install transformers and tokenizers

!pip install transformers==4.12.5 tokenizers==0.10.3



# Install PyTorch (if not already installed)

# For CPU-only version

!pip install torch



# For GPU version (CUDA 10.2)

!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 56.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
ERROR: Could not find a version that satisf

In [ ]:
# Install transformers and tokenizers

!pip install transformers==4.12.5 tokenizers==0.10.3



# Install PyTorch (if not already installed)

# For CPU-only version

!pip install torch



# For GPU version (CUDA 10.2)

!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

  Using cached transformers-4.12.5-py3-none-any.whl.metadata (56 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
Using cached transformers-4.12.5-py3-none-any.whl (3.1 MB)
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
ERROR: Could not find a version that satisfies the requirement 

In [ ]:
def seq_to_kmers(sequence, k=6):

    kmers = [sequence[x:x+k] for x in range(len(sequence) - k + 1)]

    return ' '.join(kmers)

In [ ]:
df_train['kmers'] = df_train['sequence'].apply(seq_to_kmers)
df_val['kmers'] = df_val['sequence'].apply(seq_to_kmers)
df_test['kmers'] = df_test['sequence'].apply(seq_to_kmers)

In [ ]:
df_train.head()

,signalValue,sequence,accessibility,accessibility.1,kmers
0,0.000000,CTAGTCAAACCACAGATAACTGCAGCCCAGCTGATACCAGGAGGAG...,0.0,0.0,CTAGTC TAGTCA AGTCAA GTCAAA TCAAAC CAAACC AAAC...
1,0.000000,CTTCCTTTTCAAACAGGGTAAGAGAGCTGAGGGGCCTTCCGTGGGG...,0.0,0.0,CTTCCT TTCCTT TCCTTT CCTTTT CTTTTC TTTTCA TTTC...
2,8.197100,GCTCTATAAAACAACAGGGTATAATGAGCTATCTTGGTTTCCAAAT...,1.0,1.0,GCTCTA CTCTAT TCTATA CTATAA TATAAA ATAAAA TAAA...
3,0.847976,ATTGTTGTCTGTAATGTGTTCTCATAAGTTGATTGATTCTAGATGT...,1.0,1.0,ATTGTT TTGTTG TGTTGT GTTGTC TTGTCT TGTCTG GTCT...
4,1.978610,ACCCCCTCCTCTAACCCCAGTCACCTCTGTCTCTATTCTGTGATTC...,1.0,1.0,ACCCCC CCCCCT CCCCTC CCCTCC CCTCCT CTCCTC TCCT...


In [ ]:
df_train.shape

(11019, 5)

In [ ]:
df_val.shape

(1377, 5)

In [ ]:
df_test.shape

(1378, 5)

In [ ]:
from transformers import BertTokenizer, BertModel

model_name = "zhihan1996/DNA_bert_6"

tokenizer = BertTokenizer.from_pretrained(model_name)

model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

In [ ]:
import torch.nn as nn

import torch.nn.functional as F



# Define the attention aggregator module

class AttentionAggregator(nn.Module):

    def __init__(self, embedding_dim):

        super(AttentionAggregator, self).__init__()

        self.attention = nn.Linear(embedding_dim, 1, bias=False)



    def forward(self, embeddings):

        scores = self.attention(embeddings)  # Compute attention scores

        attention_weights = F.softmax(scores, dim=0)  # Normalize scores with softmax

        weighted_embedding = torch.sum(attention_weights * embeddings, dim=0)

        return weighted_embedding

In [ ]:
# Function to get embeddings

def get_embeddings(kmers, tokenizer, model):

    embeddings_list = []

    # Create an instance of the attention aggregator

    embedding_dim = model.config.hidden_size  # Set to match your model's output size

    attentional_aggregator = AttentionAggregator(embedding_dim)

    # Tokenize all k-mers at once with padding

    inputs = tokenizer(kmers, return_tensors="pt", padding=True, truncation=True, add_special_tokens=True)

    # Extract embeddings

    with torch.no_grad():
        outputs = model(**inputs)
        # Use the CLS token's embedding for entire k-mer representation

        cls_embeddings = outputs.last_hidden_state[:, 0, :] # Extracting CLS token for each input

        embeddings_list.append(cls_embeddings)# Stack all CLS embeddings (since they are of equal shape now)

    embeddings_tensor = torch.cat(embeddings_list, dim=0)
    # Use the attention aggregator to compute a single aggregated vector

    aggregated_embedding = attentional_aggregator(embeddings_tensor)



    return aggregated_embedding.detach().numpy()

In [ ]:
all_embeddings = []


for kmer in df_val['kmers']:

    embeddings = []

    #numpy array - input it as

    #kmers = generate_kmers(sequence)

    embeddings = get_embeddings(kmer, tokenizer, model)

    all_embeddings.append(embeddings)

In [ ]:
import pandas as pd



# Convert the embeddings to a DataFrame

num_features = len(all_embeddings[0])

column_names = [f"feature_{i}" for i in range(num_features)]

embeddings_df = pd.DataFrame(all_embeddings, columns=column_names)



# Add a new column to the DataFrame

# Example: adding a label column with default values, or you can replace this with actual data

embeddings_df['accessibility'] = df_val['accessibility'] #change it to df_train and df_test as well.
#embeddings_df['signalValue'] = df_val['signalValue']


# Save the DataFrame with the new column to a CSV file

csv_filename = "/content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/dna_bert_kmer_embeddings_val.csv"

embeddings_df.to_csv(csv_filename, index=False)



print(f"Embeddings saved to {csv_filename}")

Embeddings saved to /content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/dna_bert_kmer_embeddings_val.csv


In [ ]:
all_embeddings = []


for kmer in df_train['kmers']:

    embeddings = []

    #numpy array - input it as

    #kmers = generate_kmers(sequence)

    embeddings = get_embeddings(kmer, tokenizer, model)

    all_embeddings.append(embeddings)

In [ ]:
import pandas as pd



# Convert the embeddings to a DataFrame

num_features = len(all_embeddings[0])

column_names = [f"feature_{i}" for i in range(num_features)]

embeddings_df = pd.DataFrame(all_embeddings, columns=column_names)



# Add a new column to the DataFrame

# Example: adding a label column with default values, or you can replace this with actual data

embeddings_df['accessibility'] = df_train['accessibility']
#embeddings_df['signalValue'] = df_val['signalValue']


# Save the DataFrame with the new column to a CSV file

csv_filename = "/content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/dna_bert_kmer_embeddings_train.csv"

embeddings_df.to_csv(csv_filename, index=False)



print(f"Embeddings saved to {csv_filename}")

Embeddings saved to /content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/dna_bert_kmer_embeddings_train.csv


In [ ]:
all_embeddings = []


for kmer in df_test['kmers']:

    embeddings = []

    #numpy array - input it as

    #kmers = generate_kmers(sequence)

    embeddings = get_embeddings(kmer, tokenizer, model)

    all_embeddings.append(embeddings)

In [ ]:
import pandas as pd



# Convert the embeddings to a DataFrame

num_features = len(all_embeddings[0])

column_names = [f"feature_{i}" for i in range(num_features)]

embeddings_df = pd.DataFrame(all_embeddings, columns=column_names)



# Add a new column to the DataFrame

# Example: adding a label column with default values, or you can replace this with actual data

embeddings_df['accessibility'] = df_test['accessibility'] #change it to df_train and df_test as well.
#embeddings_df['signalValue'] = df_val['signalValue']


# Save the DataFrame with the new column to a CSV file

csv_filename = "/content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/dna_bert_kmer_embeddings_test.csv"

embeddings_df.to_csv(csv_filename, index=False)



print(f"Embeddings saved to {csv_filename}")

Embeddings saved to /content/drive/MyDrive/1:1_Atharv_Gupta/Datasets/dna_bert_kmer_embeddings_test.csv
